# Digital Twin Supply Chain Simulation
Simpy model + ML bottleneck predictor (placeholder).

In [ ]:
import simpy, random, numpy as np, pandas as pd

random.seed(42)

def factory(env, out_pipe, mean_prod=100, std_prod=10):
    while True:
        yield env.timeout(1)
        qty = max(0, int(random.gauss(mean_prod, std_prod)))
        out_pipe.put(qty)

class Pipe:
    def __init__(self, env):
        self.env = env
        self.store = simpy.Store(env)
    def put(self, value):
        return self.store.put((self.env.now, value))
    def get(self):
        return self.store.get()

def dc(env, in_pipe, out_pipe, capacity=300):
    on_hand = 0
    while True:
        t, incoming = yield in_pipe.get()
        on_hand += incoming
        ship = min(on_hand, capacity)
        on_hand -= ship
        yield env.timeout(1)
        out_pipe.put(ship)

def store(env, in_pipe, demand_mean=80):
    fill_history = []
    on_hand = 0
    while True:
        t, incoming = yield in_pipe.get()
        on_hand += incoming
        d = np.random.poisson(demand_mean)
        filled = min(on_hand, d)
        on_hand -= filled
        fill_history.append((env.now, d, filled, on_hand))
        yield env.timeout(1)

env = simpy.Environment()
f2d = Pipe(env); d2s = Pipe(env)
env.process(factory(env, f2d))
env.process(dc(env, f2d, d2s))
env.process(store(env, d2s))
env.run(until=200)
print("Simulation complete.")
